In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)

In [12]:
def get_accuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
        
        print(idQueryGoal)
        print(idTable)
        print("")
        
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [3]:
indexingTables.indices.close(index='data_table_newyork')
indexingTables.indices.put_settings(index='data_table_newyork', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexingTables.indices.open(index='data_table_newyork')

{'acknowledged': True, 'shards_acknowledged': True}

In [3]:
def searchIndexing(query, tableField, topK):
    
    result= indexingTables.search(
        index="data_table_newyork", 
        body = {
        "_source": ["table_url"],
        "from" : 0,
        "size" : topK,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              #"fields": ["tablePgTitle"] 
              "fields": ["table_page_title"] 
              #"fields": [tableField]
            }
        }
    })
    
    return result

In [4]:
articles = pd.read_csv('dataset/article_data_newyork', delimiter=',', header=None)
formattedArticles = articles.iloc[:,:].values

In [15]:
formattedArticles[2:3]

array([['http://www.cnn.com/2011/US/02/27/wwi.veteran.death/index.html',
        'last living world war veteran dies',
        'washington cnn frank buckles last living world war veteran died spokesman family said sunday lawmakers monday began move ahead proposed resolutions would allow casket displayed capitol rotunda plans already works burial arlington national cemetery buckles died peacefully home natural causes early sunday morning family said statement sent cnn late sunday spokesman david dejonge buckles marked th birthday february family earlier told cnn slowed considerably since last fall according daughter susannah buckles flanagan lives family home near charles town west virginia buckles served army ambulance driver europe known great war rose rank corporal war ended his assignments included escort german prisoners war little know would someday become prisoner war world war came prominence recent years part work dejonge michigan portrait photographer undertaken project docume

In [5]:
def executeSearch(tableField, topK):

    accuracy = []

    for articles in (formattedArticles[2:3]):
    
        article_url = articles[0]
        article_page_title = articles[1]
        article_main_passage = articles[2][:6000]
        
        table_url = articles[3]
        articl_key = table_url
        
        catch = article_page_title + " " + article_main_passage 
        
        result = searchIndexing(article_page_title,tableField,topK)
    
        idRankedTables = []
    
        for hit in result['hits']['hits']:
    
            tablePgId = hit['_source']['table_url']
    
            idRankedTables.append(tablePgId)
    
        accuracy.append(get_accuracy(idRankedTables,articl_key))

    print(str(round(np.mean(accuracy),4)))

In [6]:
#tableField = ["table_page_title","table_page_content","table_caption","table_content","table_header"]
tableField = ["table_page_title","table_page_content","table_caption","table_content","table_header"]
topK = [5]

In [12]:
for k in topK:
    
    print("")
    
    for field in tableField:
        
        print("Acc@"+str(k)+" - "+field)
        
        executeSearch(field,k)


Acc@5 - table_page_title
https://en.wikipedia.org/wiki/List_of_close_election_results
https://en.wikipedia.org/wiki/List_of_close_election_results

1.0
Acc@5 - table_page_content
https://en.wikipedia.org/wiki/List_of_close_election_results
https://en.wikipedia.org/wiki/List_of_close_election_results

1.0
Acc@5 - table_caption
https://en.wikipedia.org/wiki/List_of_close_election_results
https://en.wikipedia.org/wiki/List_of_close_election_results

1.0
Acc@5 - table_content
https://en.wikipedia.org/wiki/List_of_close_election_results
https://en.wikipedia.org/wiki/List_of_close_election_results

1.0
Acc@5 - table_header
https://en.wikipedia.org/wiki/List_of_close_election_results
https://en.wikipedia.org/wiki/List_of_close_election_results

1.0


In [13]:
for k in topK:
    
    print("")
         
    print("Acc@"+str(k))
        
    executeSearch("",k)


Acc@5
https://en.wikipedia.org/wiki/List_of_last_World_War_I_veterans_by_country
https://en.wikipedia.org/wiki/List_of_last_surviving_World_War_I_veterans

https://en.wikipedia.org/wiki/List_of_last_World_War_I_veterans_by_country
https://en.wikipedia.org/wiki/List_of_last_surviving_World_War_I_veterans

https://en.wikipedia.org/wiki/List_of_last_World_War_I_veterans_by_country
https://en.wikipedia.org/wiki/List_of_last_World_War_I_veterans_by_country

1.0
